# Our Environment

In [1]:
import torch
import tensorflow as tf
import os

print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Version:", torch.version.cuda)

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2023-09-07 13:02:54.587348: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 13:02:56.051974: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


PyTorch Version: 2.0.1+cu117
CUDA Available: True
CUDA Version: 11.7
Num GPUs Available:  4


In [4]:
import ray
from ray import air, tune
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.algorithms.ppo import PPOConfig

from custom_env import CustomEnvironment
from config import run_config

## The RLlib configuration
class Args:
    def __init__(self):
        self.run = "PPO"
        self.framework = "torch" # "tf2" or "torch"
        self.stop_iters = 5
        self.stop_timesteps = 100000
        self.stop_reward = 0.1
        self.as_test = False

args = Args()

## Generate the configuration
ray.init()
env = CustomEnvironment(run_config["env"])

config = (
    PPOConfig()
    .rollouts(rollout_fragment_length="auto", num_rollout_workers=1)
    .environment(CustomEnvironment, env_config=run_config["env"])
    .framework(args.framework)
    .training(num_sgd_iter=10, sgd_minibatch_size=256, train_batch_size=4000)
    .multi_agent(
        policies= {
            "prey": PolicySpec(
                policy_class=None,  # infer automatically from Algorithm
                observation_space=env.observation_space[0],  # if None infer automatically from env
                action_space=env.action_space[0],  # if None infer automatically from env
                config={"gamma": 0.85},  # use main config plus <- this override here
            ),
            "predator": PolicySpec(
                policy_class=None,
                observation_space=env.observation_space[0],
                action_space=env.action_space[0],
                config={"gamma": 0.85},
            ),
        },
        policy_mapping_fn = lambda id, *arg, **karg: "prey" if env.agents[id].agent_type == 0 else "predator",
        policies_to_train=["prey", "predator"]
    )
    .rl_module(_enable_rl_module_api=True)
    .training(_enable_learner_api=True)
    .resources(
        num_gpus_per_learner_worker=1,
        num_cpus_per_worker = 4,
        num_learner_workers= 4
    )
)


stop = {
    "training_iteration": args.stop_iters,
    "timesteps_total": args.stop_timesteps,
    "episode_reward_mean": args.stop_reward,
}

## Run the experiemnt    

tuner = tune.Tuner(
    args.run,
    param_space=config.to_dict(),
    run_config=air.RunConfig(stop=stop, verbose=3),
)
results = tuner.fit()

if args.as_test:
    print("Checking if learning goals were achieved")
    check_learning_achieved(results, args.stop_reward)
ray.shutdown()



2023-09-07 13:49:03,306	INFO worker.py:1621 -- Started a local Ray instance.
2023-09-07 13:49:13,063	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-09-07 13:49:13,078	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-09-07 13:49:13,087	WARNING algorithm_config.

2023-09-07 13:49:13,116	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=39835) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=39835) 2023-09-07 13:49:22,641	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config=

(_WrappedExecutable pid=40168) [E ProcessGroupNCCL.cpp:828] [Rank 0] Watchdog caught collective operation timeout: WorkNCCL(SeqNum=1869, OpType=ALLREDUCE, Timeout(ms)=1800000) ran for 1800475 milliseconds before timing out.
(_WrappedExecutable pid=40168) [E ProcessGroupNCCL.cpp:455] Some NCCL operations have failed or timed out. Due to the asynchronous nature of CUDA kernels, subsequent GPU operations might run on corrupted/incomplete data.
(_WrappedExecutable pid=40168) [E ProcessGroupNCCL.cpp:460] To avoid data inconsistency, we are taking the entire process down.
(_WrappedExecutable pid=40168) [2023-09-07 14:22:34,820 E 40168 40567] logging.cc:97: Unhandled exception: St13runtime_error. what(): [Rank 0] Watchdog caught collective operation timeout: WorkNCCL(SeqNum=1869, OpType=ALLREDUCE, Timeout(ms)=1800000) ran for 1800475 milliseconds before timing out.
(_WrappedExecutable pid=40168) [2023-09-07 14:22:34,912 E 40168 40567] logging.cc:104: Stack trace: 
(_WrappedExecutable pid=4016